In [1]:
import pandas as pd
from statsmodels.stats.multitest import multipletests
import seaborn as sns
import os
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import decoupler as dc
import mygene
import seaborn as sns
from scipy.stats import wilcoxon
import matplotlib.pyplot as plt
from statannotations.Annotator import Annotator

In [2]:
collectri = dc.op.collectri(organism="human")
print(len(collectri))

42990


In [113]:
tissue = "Breast"
tissue_path = f"gtex_fdr_results/{tissue}/random_100_with_shuffled_occurences/fdr_grn_nontf_100_numtf_-1.csv"
tissue_df = pd.read_csv(tissue_path)

# Filter to top 1000 highest expressed targets.
print("Num edges before filtering: ", len(tissue_df))
high_exp_file = os.path.join("gtex", tissue, f"{tissue}_highly_expressed_targets.csv")
high_df = pd.read_csv(high_exp_file)
top_targets = list(high_df['target'])[:1000]
tissue_df = tissue_df[tissue_df['target'].isin(top_targets)]
print("Num edges after subsetting to highest expressed genes: ", len(tissue_df))

# Filter top 10% of targets for each TF.
density = 1.0
tissue_df = (
    tissue_df.groupby("TF", group_keys=False)
      .apply(lambda g: g.nlargest(max(1, int(len(g) * density)), "importance"))
)
print(f"Num edges after filtering to top {density} edges per TF: ", len(tissue_df))

Num edges before filtering:  1746068
Num edges after subsetting to highest expressed genes:  109710
Num edges after filtering to top 1.0 edges per TF:  109710


/tmp/ipykernel_3772485/317269442.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: g.nlargest(max(1, int(len(g) * density)), "importance"))


In [114]:
# Translate ensemble gene IDs to gene symbols.
to_translate = list(set(tissue_df['TF']).union(set(tissue_df['target'])))

mg = mygene.MyGeneInfo()
# Query Ensembl IDs and return gene symbols
result = mg.querymany(to_translate, scopes='ensembl.gene', fields='symbol', species='human')

# Convert to a mapping dictionary
ensembl_to_symbol = {r['query']: r.get('symbol', None) for r in result}
# Filter out genes that have no matching gene symbol.
failure_genes = [x for x,y in ensembl_to_symbol.items() if not y]
success_genes = list(set(to_translate) - set(failure_genes))
# Subset GRN to genes that have matching translation.
tissue_df = tissue_df[tissue_df['TF'].isin(success_genes)]
tissue_df = tissue_df[tissue_df['target'].isin(success_genes)]
# Apply the translation to both columns
tissue_df['TF'] = tissue_df['TF'].map(ensembl_to_symbol)
tissue_df['target'] = tissue_df['target'].map(ensembl_to_symbol)


2025-10-28 10:02:01 | [WARNING] Input sequence provided is already in string format. No operation performed
2025-10-28 10:02:01 | [WARNING] Input sequence provided is already in string format. No operation performed
2025-10-28 10:02:01 | [INFO] querying 1-1000 ...
2025-10-28 10:02:03 | [INFO] HTTP Request: POST https://mygene.info/v3/query/ "HTTP/1.1 200 OK"
2025-10-28 10:02:04 | [INFO] querying 1001-2000 ...
2025-10-28 10:02:05 | [INFO] HTTP Request: POST https://mygene.info/v3/query/ "HTTP/1.1 200 OK"
2025-10-28 10:02:06 | [INFO] querying 2001-2596 ...
2025-10-28 10:02:07 | [INFO] HTTP Request: POST https://mygene.info/v3/query/ "HTTP/1.1 200 OK"
2025-10-28 10:02:08 | [INFO] Finished.
2025-10-28 10:02:08 | [WARNING] 1 input query terms found no hit:	['ENSG00000189144']
2025-10-28 10:02:08 | [INFO] Pass "returnall=True" to return complete lists of duplicate or missing query terms.


In [121]:
# FDR control of remaining edges.
_, pvals_bh, _, _ = multipletests(tissue_df['pvalue'], method='fdr_bh')
tissue_df['pvalue_bh'] = pvals_bh
print("Minimal occuring p-value: ", min(tissue_df['pvalue_bh']))
signif_df = tissue_df[tissue_df['pvalue_bh']<=0.05].copy()
print("Number of total edges: ", len(tissue_df))
print("Number of signif. edges: ", len(signif_df))

Minimal occuring p-value:  0.012201273114412216
Number of total edges:  109677
Number of signif. edges:  20364


In [120]:
# Compute intersection of non-thresholded and thresholded edges.
all_edges = set(zip(tissue_df['TF'], tissue_df['target']))
signif_edges = set(zip(signif_df['TF'], signif_df['target']))
collectri_edges = set(zip(collectri['source'], collectri['target']))

# Find the intersection
all_common = all_edges & collectri_edges
signif_common = signif_edges & collectri_edges

# Compute precisions.
precision_all = len(all_common) / len(all_edges)
precision_signif = len(signif_common) / len(signif_edges)

print("Precision of all edges: ", precision_all)
print("Precision of significant edges: ", precision_signif)
print("Total amount of all edges: ", len(all_common))
print("Total amound of signif edges: ", len(signif_common))

Precision of all edges:  0.0008844151462932064
Precision of significant edges:  0.0015590200445434299
Total amount of all edges:  97
Total amound of signif edges:  14


In [3]:
tissues = ["Breast", "Kidney", "Testis"]
res_dict = {'tissue': [], 'type' : [], 'precision' : [], 'recall' : []}

for tissue in tissues:
    print("Processing tissue ", tissue)
    for i in range(10):
        print("Num run = ", i)
        tissue_path = f"gtex_fdr_results/{tissue}/random_robustness_check/fdr_grn_nontf_100_numtf_-1_input_{i}.csv"
        tissue_df = pd.read_csv(tissue_path)

        # Filter to top 1000 highest expressed targets.
        print("Num edges before filtering: ", len(tissue_df))
        high_exp_file = os.path.join("gtex", tissue, f"{tissue}_highly_expressed_targets.csv")
        high_df = pd.read_csv(high_exp_file)
        top_targets = list(high_df['target'])[:1000]
        tissue_df = tissue_df[tissue_df['target'].isin(top_targets)]
        print("Num edges after subsetting to highest expressed genes: ", len(tissue_df))

        # Filter top 10% of targets for each TF.
        density = 1.0
        tissue_df = (
            tissue_df.groupby("TF", group_keys=False)
            .apply(lambda g: g.nlargest(max(1, int(len(g) * density)), "importance"))
        )
        print(f"Num edges after filtering to top {density} edges per TF: ", len(tissue_df))
        
        # Translate ensemble gene IDs to gene symbols.
        to_translate = list(set(tissue_df['TF']).union(set(tissue_df['target'])))

        mg = mygene.MyGeneInfo()
        # Query Ensembl IDs and return gene symbols
        result = mg.querymany(to_translate, scopes='ensembl.gene', fields='symbol', species='human')

        # Convert to a mapping dictionary
        ensembl_to_symbol = {r['query']: r.get('symbol', None) for r in result}
        # Filter out genes that have no matching gene symbol.
        failure_genes = [x for x,y in ensembl_to_symbol.items() if not y]
        success_genes = list(set(to_translate) - set(failure_genes))
        # Subset GRN to genes that have matching translation.
        tissue_df = tissue_df[tissue_df['TF'].isin(success_genes)]
        tissue_df = tissue_df[tissue_df['target'].isin(success_genes)]
        # Apply the translation to both columns
        tissue_df['TF'] = tissue_df['TF'].map(ensembl_to_symbol)
        tissue_df['target'] = tissue_df['target'].map(ensembl_to_symbol)
        
        # FDR control of remaining edges.
        _, pvals_bh, _, _ = multipletests(tissue_df['pvalue'], method='fdr_bh')
        tissue_df['pvalue_bh'] = pvals_bh
        signif_df = tissue_df[tissue_df['pvalue_bh']<=0.05].copy()
        print("Number of total edges: ", len(tissue_df))
        print("Number of signif. edges: ", len(signif_df))
        
        # Compute intersection of non-thresholded and thresholded edges.
        all_edges = set(zip(tissue_df['TF'], tissue_df['target']))
        signif_edges = set(zip(signif_df['TF'], signif_df['target']))
        collectri_edges = set(zip(collectri['source'], collectri['target']))
        num_collectri_edges = len(collectri_edges)

        # Find the intersection
        all_common = all_edges & collectri_edges
        signif_common = signif_edges & collectri_edges

        # Compute precisions.
        precision_all = len(all_common) / len(all_edges)
        precision_signif = len(signif_common) / len(signif_edges)
        
        recall_all = len(all_common) / num_collectri_edges
        recall_signif = len(signif_common) / num_collectri_edges
        
        res_dict['tissue'].append(tissue)
        res_dict['type'].append('all')
        res_dict['precision'].append(precision_all)
        res_dict['recall'].append(recall_all)
        
        res_dict['tissue'].append(tissue)
        res_dict['type'].append('signif')
        res_dict['precision'].append(precision_signif)
        res_dict['recall'].append(recall_signif)
    
    

Processing tissue  Breast
Num run =  0
Num edges before filtering:  1746068
Num edges after subsetting to highest expressed genes:  109710


/tmp/ipykernel_3388537/586548921.py:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: g.nlargest(max(1, int(len(g) * density)), "importance"))
2025-11-07 16:53:37 | [WARNING] Input sequence provided is already in string format. No operation performed
2025-11-07 16:53:37 | [WARNING] Input sequence provided is already in string format. No operation performed
2025-11-07 16:53:37 | [INFO] querying 1-1000 ...


Num edges after filtering to top 1.0 edges per TF:  109710


2025-11-07 16:53:41 | [INFO] HTTP Request: POST https://mygene.info/v3/query/ "HTTP/1.1 200 OK"
2025-11-07 16:53:42 | [INFO] querying 1001-2000 ...
2025-11-07 16:53:43 | [INFO] HTTP Request: POST https://mygene.info/v3/query/ "HTTP/1.1 200 OK"
2025-11-07 16:53:44 | [INFO] querying 2001-2596 ...
2025-11-07 16:53:45 | [INFO] HTTP Request: POST https://mygene.info/v3/query/ "HTTP/1.1 200 OK"
2025-11-07 16:53:46 | [INFO] Finished.
2025-11-07 16:53:46 | [WARNING] 1 input query terms found no hit:	['ENSG00000189144']
2025-11-07 16:53:46 | [INFO] Pass "returnall=True" to return complete lists of duplicate or missing query terms.


Number of total edges:  109677
Number of signif. edges:  20317
Num run =  1
Num edges before filtering:  1800126
Num edges after subsetting to highest expressed genes:  115173


...

Number of total edges:  111952
Number of signif. edges:  23477
Num run =  9
Num edges before filtering:  2077780
Num edges after subsetting to highest expressed genes:  115130


/tmp/ipykernel_3388537/586548921.py:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: g.nlargest(max(1, int(len(g) * density)), "importance"))
2025-11-07 16:58:21 | [WARNING] Input sequence provided is already in string format. No operation performed
2025-11-07 16:58:21 | [WARNING] Input sequence provided is already in string format. No operation performed
2025-11-07 16:58:21 | [INFO] querying 1-1000 ...


Num edges after filtering to top 1.0 edges per TF:  115130


2025-11-07 16:58:22 | [INFO] HTTP Request: POST https://mygene.info/v3/query/ "HTTP/1.1 200 OK"
2025-11-07 16:58:23 | [INFO] querying 1001-2000 ...
2025-11-07 16:58:25 | [INFO] HTTP Request: POST https://mygene.info/v3/query/ "HTTP/1.1 200 OK"
2025-11-07 16:58:26 | [INFO] querying 2001-2827 ...
2025-11-07 16:58:27 | [INFO] HTTP Request: POST https://mygene.info/v3/query/ "HTTP/1.1 200 OK"
2025-11-07 16:58:28 | [INFO] Finished.
2025-11-07 16:58:28 | [WARNING] 1 input query terms found no hit:	['ENSG00000189144']
2025-11-07 16:58:28 | [INFO] Pass "returnall=True" to return complete lists of duplicate or missing query terms.


Number of total edges:  115103
Number of signif. edges:  24277


In [4]:
result_df = pd.DataFrame(res_dict)
result_df.to_csv("precision_recall_results_against_collectri.csv", index=False)